In [ ]:
from models.DistMult import DistMult
from utils.loaders import load_data
import pickle
import torch
import numpy as np

In [ ]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)

entities = len(lookup)

In [ ]:
objects, subjects, relationships  = load_data(data, 100)
obj = torch.LongTensor(objects[0])
rel = torch.LongTensor(relationships[0])
subj = torch.squeeze(torch.LongTensor(subjects[0]))

In [ ]:
model = DistMult(num_entities = entities, embedding_dim=100)
print(model)

In [ ]:
pred = model(e1 = obj, rel = rel)

In [ ]:
print(model.loss(pred, subj))

In [ ]:
# best way to test your system is to train on 100 examples and tests on the 100 same examples - if your model doesn't overfit after a bundle of epochs there is a mistake. 
# knowledge base models take a long long time to converge. They can take 100s of epochs to converge. YOU SHOULD SHUFFLE YOUR DATA EVERY TIME!!! Don't have the same stuff in each batch. 
# learn to use the data loader for shuffling. Alternatively do it yourself.  

In [ ]:
# with BCE you can do label smoothing. 

In [ ]:
out.size()
#okay this makes sense its spitting out a prediction!!! 

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=0.995)

In [ ]:
model.train()